# The LUNA/BUSD Order Book 🌍🌕

# 🔥🔥🔥 is on Fire! 🔥🔥🔥

In [54]:
import ipywidgets as widgets
import pandas as pd
from ipywidgets import Layout
import requests

tab = widgets.HTML(value="")
btn = widgets.Button(description='Run',button_style='', tooltip='Run')
btn.style.button_color = 'lightgreen'

In [55]:
def style_pipe(df):
    s= df.style.hide_index().hide_columns(['side'])

    df_mask = df.copy(deep=True)
    df_mask[df['side'] == 'bid'] = 'bid'
    df_mask[df['side'] == 'ask'] = 'ask'

    s.set_table_styles([
        {'selector': 'th:not(.index_name)','props':'text-align:center; color:grey;background-color:#373737'},
        {'selector': 'td','props':'text-align:center; background-color:#2d2d2d;font-weight:bold'},
        {'selector':'.bid','props':'color:green'},
        {'selector':'.ask','props':'color:red'},
    ],overwrite=False)

    s.set_td_classes(df_mask)   

    idxs = df.index[df['side'] == 'bid'].tolist()
    idx = pd.IndexSlice
    s.bar(subset=idx[:idxs[-1],'quantity'], color='#dbeac5')
    s.bar(subset=idx[idxs[-1]+1:,'quantity'], color='#ff9090')

    return s 

In [56]:
hBox = widgets.HBox([btn],layout=Layout(justify_content= 'center'))


def run_dash(b):
    count=0

    hBox.children = [tab]

    while count<100:
        #time.sleep(5)
        r = requests.get("https://api.binance.com/api/v3/depth",params=dict(symbol="LUNABUSD"))
        response = r.json()

        df_bid = pd.DataFrame.from_records(response['bids'], columns=['price', 'quantity']).astype('float')
        df_bid['side'] = 'bid'

        df_ask = pd.DataFrame.from_records(response['asks'], columns=['price', 'quantity']).astype('float')
        df_ask['side'] = 'ask'

        df = pd.concat([df_bid.sort_values(by=['price'])[-5:], df_ask[:5]], ignore_index=True)
        count=count+1

        #hBox.children[1].value = style_pipe(df).to_html()

        tab.value=style_pipe(df).to_html()
    
btn.on_click(run_dash)


In [57]:
%%html
<style>
    .box_style{
        width:100%;
        border : 0px solid red;
        height: auto;
        background-color:#2d2d2d;
    }
</style>

In [58]:
hBox.add_class("box_style")
hBox